In [1]:
import org.springframework.ai.transformers.TransformersEmbeddingModel

val embeddingModel = TransformersEmbeddingModel()
embeddingModel.setModelResource("file:resources/model/bge-large-en-v1.5/model.onnx")
embeddingModel.setTokenizerResource("file:resources/model/bge-large-en-v1.5/tokenizer.json")
embeddingModel.afterPropertiesSet()

In [2]:
import dev.raphaeldelio.*
import org.springframework.ai.vectorstore.redis.RedisVectorStore
import org.springframework.ai.vectorstore.redis.RedisVectorStore.MetadataField
import redis.clients.jedis.search.Schema.FieldType

val redisVectorStore = RedisVectorStore.builder(jedisPooled, embeddingModel)
    .indexName("semanticCacheIdx")
    .contentFieldName("text")
    .embeddingFieldName("textEmbedding")
    .metadataFields(
        MetadataField("answer", FieldType.TEXT),
    )
    .prefix("semanticcache:")
    .initializeSchema(true)
    .vectorAlgorithm(RedisVectorStore.Algorithm.FLAT)
    .build()
redisVectorStore.afterPropertiesSet()

In [3]:
import org.springframework.ai.document.Document
import java.util.UUID

fun insertIntoCache(post: String, answer: String) {
    redisVectorStore.add(listOf(createDocument(post, answer)))
}

fun createDocument(post: String, answer: String): Document {
    return Document(
        UUID.randomUUID().toString(),
        post,
        mapOf("answer" to answer),
    )
}

In [4]:
fun getFromCache(post: String): String {
    val result = redisVectorStore.similaritySearch(post)?.first()
    val score = redisVectorStore.similaritySearch(post)?.first()?.score ?: 0.0
    if (score > 0.9) {
        return result?.metadata?.get("answer").toString()
    } else {
        return ""
    }
}

In [5]:
insertIntoCache("What's trending right now?", "This, this, and that")

In [6]:
getFromCache("What's trending today?")

This, this, and that

In [7]:
getFromCache("What are people saying about ChatGPT?")